In [1]:
import time

def toi32(x):
    x%=(2**32)
    if x>=2**31:
        return x-2**32
    return x

funcTable={
    "add":(2,(lambda a,b: a+b)),
    "sub":(2,(lambda a,b: a-b)),
    "mul":(2,(lambda a,b: a*b)),
    "rem_s":(2,(lambda a,b: a%b)),
    "and":(2,(lambda a,b: a&b)),
    "or":(2,(lambda a,b: a|b)),
    "eq":(2,(lambda a,b: int(a==b))),
    "ne":(2,(lambda a,b: int(a!=b))),
    "lt_s":(2,(lambda a,b: int(a<b))),
    "le_s":(2,(lambda a,b: int(a<=b))),
    "gt_s":(2,(lambda a,b: int(a>b))),
    "ge_s":(2,(lambda a,b: int(a>=b))),
    "shl":(2,(lambda a,b: int(a<<b))),
    "eqz":(1,(lambda a: int(a==0))),
}

class ASMRunner:
    CODE=[]
    MEMORY=[]
    LOCAL=[]
    GLOBAL=[]
    LABEL={}
    STACK=[]
    IF_TABLE={}
    def __init__(self):
        return
    def loadCode(self,codef):
        self.CODE=[]
        self.MEMORY=[]
        self.LOCAL=[]
        self.GLOBAL=[]
        self.LABEL={}
        self.STACK=[]
        self.IF_TABLE={}
        x=codef.split('\n')
        for a in x:
            self.CODE.append(a.strip())
        # self.CODE: 逐行读取的命令，是list
        self.scanCode()
    def scanCode(self):
        ifs=[]
        for i in range(len(self.CODE)):
            nl=self.CODE[i]
            cdl=nl.split(' ')
            if cdl[0]=="loop":
                try:
                    gid=int(cdl[1][6:]) # 读出来loop的label
                    self.LABEL[gid]=i   # 即，该label在字典上对应这个loop
                except:
                    raise Exception("unknow identifier at line %d:"%i)
            if cdl[0]=="if":
                ifs.append(i)
            if cdl[0]=="else":
                try:
                    itt=ifs[-1]
                    ifs.pop()
                    self.IF_TABLE[itt]=i
                except:
                    raise Exception("invalid if block at line %d:"%i)
                ifs.append(i)
            if cdl[0]=="end":
                if len(cdl)==2:
                    try:
                        gid=int(cdl[1][6:])
                        if not gid in self.LABEL:
                            self.LABEL[gid]=i
                    except:
                        raise Exception("unknow identifier at line %d:"%i)
                    continue
                try:
                    itt=ifs[-1]
                    ifs.pop()
                    self.IF_TABLE[itt]=i
                except:
                    raise Exception("invalid if block at line %d:"%i)
        for x in ifs:
            raise Exception("invalid if block at line %d:"%x)

    def getStack(self):
        if len(self.STACK)==0:
            raise Exception("stack error at line %d"%(self.pc+1))
            return 0
        x=self.STACK[-1]
        self.STACK.pop()
        return x

    def getMem(self,idt,pos):
        if idt==0:
            mem=self.LOCAL
        elif idt==1:
            mem=self.GLOBAL
        else:
            mem=self.MEMORY
        if len(mem)<=pos:
            ngl=pos-len(mem)+1
            mem.extend([0]*ngl)
        return mem[pos]
    
    def setMem(self,idt,pos,val):
        if idt==0:
            mem=self.LOCAL
        elif idt==1:
            mem=self.GLOBAL
        else:
            mem=self.MEMORY
        if len(mem)<=pos:
            ngl=pos-len(mem)+1
            mem.extend([0]*ngl)
        mem[pos]=val

    def run(self):
        self.pc=0
        while True:
            nl=self.CODE[self.pc]
            if len(nl)==0:
                self.pc+=1
                continue
            cdl=nl.split(' ')
            if cdl[0] in ["loop","block","end","else","call"]:
                self.pc+=1
                continue
            if cdl[0]=="return":
                return self.getStack()
            if cdl[0]=="br":
                try:
                    itt=int(cdl[1][6:])
                    self.pc=self.LABEL[itt]+1
                except:
                    raise Exception("unknow identifier at line %d:"%(self.pc+1))
                continue
            if cdl[0]=="if":
                ifv=self.getStack()
                if ifv==0:
                    self.pc=self.IF_TABLE[self.pc]+1
                else:
                    self.pc+=1
                continue
            pl=cdl[0].split('.')
            if len(pl)!=2:
                raise Exception("unknow identifier at line %d:"%(self.pc+1))
            if pl[0] not in ["local","global","i32"]:
                raise Exception("unknow identifier at line %d:"%(self.pc+1))
            if pl[0]=="local":
                if pl[1] not in ["set","get"]:
                    raise Exception("unknow identifier at line %d:"%(self.pc+1))
                try:
                    itt=int(cdl[1][4:])
                except:
                    raise Exception("unknow identifier at line %d:"%(self.pc+1))
                if pl[1]=="set":
                    va=self.getStack()
                    self.setMem(0,itt,va)
                else:
                    self.STACK.append(self.getMem(0,itt))
                self.pc+=1
                continue
            if pl[0]=="global":
                if pl[1] not in ["set","get"]:
                    raise Exception("unknow identifier at line %d:"%(self.pc+1))
                try:
                    itt=int(cdl[1][7:])
                except:
                    raise Exception("unknow identifier at line %d:"%(self.pc+1))
                if pl[1]=="set":
                    va=self.getStack()
                    self.setMem(1,itt,va)
                else:
                    self.STACK.append(self.getMem(1,itt))
                self.pc+=1
                continue
            if pl[1]=="load":
                pos=self.getStack()
                val=self.getMem(2,pos)
                self.STACK.append(val)
                self.pc+=1
                continue
            if pl[1]=="store":
                va=self.getStack()
                pos=self.getStack()
                self.setMem(2,pos,va)
                self.pc+=1
                continue
            if pl[1]=="const":
                try:
                    itt=int(cdl[1])
                except:
                    raise Exception("unknow identifier at line %d:"%(self.pc+1))
                self.STACK.append(itt)
                self.pc+=1
                continue
            if not pl[1] in funcTable:
                raise Exception("unknow identifier at line %d:"%(self.pc+1))
            cb=self.getStack()
            if funcTable[pl[1]][0]==1:
                self.STACK.append(toi32(funcTable[pl[1]][1](cb)))
                self.pc+=1
                continue
            ca=self.getStack()
            self.STACK.append(toi32(funcTable[pl[1]][1](ca,cb)))
            self.pc+=1
        return 0

class Problem:
    def __init__(self):
        f=open("asm.txt")
        code=f.read()
        f.close()
        self.runner=ASMRunner()
        self.runner.loadCode(code)
        self.runner.setMem(1,18,4982)
        self.runner.setMem(1,19,5400)
        self.runner.setMem(1,20,4986)
        self.runner.setMem(1,21,5000)
        key='.q~03QKLNSp"s6AQtEW<=MNv9(ZMYntg2N9hSe5=k'
        self.putBytes(5400,key.encode())
        self.runner.setMem(2,4986,len(key))
        return
    def putBytes(self,pos,bts):
        for i in range(len(bts)):
            self.runner.setMem(2,pos+4*i,bts[i])
    def run(self,inv):
        length=len(inv)
        self.runner.setMem(2,4982,length)
        bts=inv.encode()
        self.putBytes(5000,bts)
        result=self.runner.run()
        if result==-1:
            print("ERROR!")
        elif result==0:
            print("WRONG!")
        else:
            print("RIGHT!")


In [2]:
obj=Problem()
loc = obj.runner.LOCAL
glo = obj.runner.GLOBAL
mem = obj.runner.MEMORY

In [3]:
loc = [0]*1000

In [4]:
glo

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4982, 5400, 4986, 5000]

### mem 的5000~5000+4len(flag)应填充为flag.encode()，mem[4982]=length

In [5]:
stk = []

In [6]:
glo[16]=16

In [7]:
loc[114] = 0
loc[113] = 0
loc[111] = 0
loc[0] = 0
loc[111] = 114514
mem[3776] = 0

In [8]:
stk

[]

label0 label1

In [ ]:
loop, label1

while True:
    '''
    loc[1] = mem[3776]
    loc[23] = (loc[1]<96)
    '''
    if mem[3776]>=96:
        break

    '''
    loc[34] = mem[3776]
    loc[45] = mem[3776]
    loc[56] = 1952 + loc[45]<<2
    mem[loc[56]] = loc[34]    
    '''
    mem[1952+(mem[3776]<<2)] = mem[3776]

    '''
    loc[67] = mem[3776]
    loc[78] = mem[3776]+1
    mem[3776] = loc[78]
    '''
    mem[3776] = mem[3776]+1

In [9]:
for i in range(96):
    mem[1952+(i*4)]=i
mem[3776] = 96

label2 label3

In [ ]:
loop label2

mem[3776] = 1

while True:
    '''
    loc[89]=mem[3776]
    loc[100] = mem[3776]<96
    '''
    if mem[3776]>=96:
        break
    
    '''
    loc[2] = loc[111]
    loc[13] = loc[111]*1919
    loc[15] = loc[111]*1919+7
    loc[16] = loc[15]%334363       # & -1
    loc[17] = loc[16]
    loc[111] = loc[16]
    
    loc[18] = mem[3776]
    loc[19] = loc[17] % loc[18]
    loc[113] = loc[19]
    
    loc[20] = loc[113]
    loc[21] = 1952 + 4*loc[20]
    loc[22] = mem[loc[21]]
    loc[114] = loc[22]
    
    loc[24] = mem[3776]
    loc[25] = 1952 + mem[3776]*4
    loc[26] = mem[loc[25]]
    loc[27] = loc[113]
    loc[28] = 1952+loc[27]*4
    mem[loc[28]] = mem[loc[26]]
    
    loc[29] = loc[114]
    loc[30] = mem[3776]
    loc[31] = 1952+loc[30]*4
    mem[loc[31]] = loc[29]
    
    loc[32] = mem[3776]
    loc[33] = mem[3776]+1
    mem[3776] = loc[33]
    '''
    
    
    loc[111] = (loc[111]*1919+7)%334363
    loc[113] = loc[111]%mem[3776]
    # loc[114] = mem[1952+4*loc[113]]
    loc[26] = mem[1952+mem[3776]*4]
    loc[28] = 1952+4*loc[113]
    mem[loc[28]] = mem[loc[26]]
    mem[1952+4*mem[3776]] = mem[1952+4*loc[113]]
    mem[3776] += 1
    

In [10]:
for i in range(1,96):
    loc[111] = (loc[111]*1919+7)%334363
    loc[113] = loc[111]%i
    pos1 = 1952+4*loc[113]
    pos2 = 1952+i*4
    tmp = mem[pos1]
    mem[pos1]= mem[pos2]
    mem[pos2] = tmp
    print("mem[%d]=%d, mem[%d]=%d"%(pos1,mem[pos1],pos2,mem[pos2]))

mem[1952]=1, mem[1956]=0
mem[1952]=2, mem[1960]=1
mem[1960]=3, mem[1964]=1
mem[1956]=4, mem[1968]=0
mem[1968]=5, mem[1972]=0
mem[1964]=6, mem[1976]=1
mem[1956]=7, mem[1980]=4
mem[1980]=8, mem[1984]=4
mem[1972]=9, mem[1988]=0
mem[1964]=10, mem[1992]=6
mem[1960]=11, mem[1996]=3
mem[1952]=12, mem[2000]=2
mem[1968]=13, mem[2004]=5
mem[1972]=14, mem[2008]=9
mem[1960]=15, mem[2012]=11
mem[2012]=16, mem[2016]=11
mem[1956]=17, mem[2020]=7
mem[1968]=18, mem[2024]=13
mem[1972]=19, mem[2028]=14
mem[2020]=20, mem[2032]=7
mem[1952]=21, mem[2036]=12
mem[1952]=22, mem[2040]=21
mem[2032]=23, mem[2044]=7
mem[1968]=24, mem[2048]=18
mem[1956]=25, mem[2052]=17
mem[2000]=26, mem[2056]=2
mem[1968]=27, mem[2060]=24
mem[1956]=28, mem[2064]=25
mem[2044]=29, mem[2068]=7
mem[1960]=30, mem[2072]=15
mem[1964]=31, mem[2076]=10
mem[2076]=32, mem[2080]=10
mem[1976]=33, mem[2084]=1
mem[2080]=34, mem[2088]=10
mem[1992]=35, mem[2092]=6
mem[2020]=36, mem[2096]=20
mem[2020]=37, mem[2100]=36
mem[1976]=38, mem[2104]=33
mem[

label4 label5

In [ ]:
loop label 5

mem[3776] = 0
while True:
    
    '''
    loc[35] = mem[3776]
    loc[36] = mem[4982]
    '''
    if mem[3776]>=41: # 长度
        break
    
    loc[113] = mem[mem[3776]*4+5000] - 32
    if loc[113]<0 or loc[113]>=96:
        loc[115] = 10
        break
    
    loc[51] = mem[1952+loc[113]*4]+mem[3776]
    mem[1152+4*mem[3776]] = loc[51]%96 +32
    
    mem[3776] = mem[3776]+1
    
    '''
    loc[39] = mem[3776]*4+5000
    loc[40] = mem[mem[3776]*4+5000]
    loc[41] = loc[40] - 32
    loc[113] = loc[41]
    loc[42] = loc[41]
    
    loc[43] = ( loc[41]<0 )
    loc[46] = (loc[113] >= 96)
    loc[112] = loc[43] or loc[46]
    
    if loc[41]<0 or loc[113]>=96:
        loc[115] = 10
        break  

    loc[48] = 1952+loc[113]*4
    loc[49] = mem[1952+loc[113]*4]
    
    loc[51] = loc[49]+mem[3776]
    loc[53] = loc[51]%96 +32
    loc[55] = 1152+4*mem[3776]
    mem[1152+4*mem[3776]] = loc[53]
    
    mem[3776] = mem[3776]+1
    
    '''

In [ ]:
for i in range(41):
    
    loc[113] = mem[i*4+5000] - 32   # flag[i]-32
    
    # flag取值：[32,127]
    if loc[113]<0 or loc[113]>=96:
        print("gg")
        break
    
    loc[51] = mem[1952+loc[113]*4]+i # 取值自1952~2332
    mem[1152+4*i] = loc[51]%96 +32   # 影响1152~1312
    

label6 label7

In [ ]:
mem[3776] = 0
while True:
    if mem[3776] >= 41:
        break
        
    '''
    
    mem[2336+4*mem[3776]] = mem[3776]
    
    mem[3776] = mem[3776]+1
    
    '''

In [ ]:
for i in range(41):
    mem[2336+4*i] = i

label8 label9

In [ ]:
print(loc[111])

In [ ]:
mem[3776] = 1
while True:
    if mem[3776] >= 41:
        break
        
    '''
    loc[71l = loc[111]
    loc[111] = (loc[71]*1919+7)%334363
    
    loc[77] = oc[111] % mem[3776]
    loc[113] = loc[77]
    loc[79] = loc[77]
    
    loc[81] = mem[2336+4*loc[79]]
    loc[114] = loc[81]
    
    loc[84] = mem[2336+4*mem[3776]]
    loc[85] = loc[113]
    
    mem[2336+4*loc[85]] = loc[84]
    
    loc[87] = loc[114]
    
    mem[2336+4*mem[3776]] = loc[87]
    
    mem[3776]=mem[3776]+1
    '''

In [ ]:
for i in range(1, 41):
    loc[111] = (loc[111]*1919+7)%334363
    loc[113] = loc[111] % i
    
    pos1 = 2336+4*loc[113]
    pos2 = 2336+i*4
    tmp = mem[pos1]
    mem[pos1]= mem[pos2]
    mem[pos2] = tmp
    
    print("mem[%d]=%d, mem[%d]=%d"%(pos1,mem[pos1],pos2,mem[pos2]))

# 影响 2336~2496
'''
    loc[111] = (loc[111]*1919+7)%334363
    
    loc[113] = loc[111] % mem[3776]
    
    mem[2336+4*loc[113]] = mem[2336+4*mem[3776]]
    
    mem[2336+4*mem[3776]] = mem[2336+4*loc[113]]
    
    mem[3776]=mem[3776]+1
    '''

label10 label11

In [ ]:
mem[3776] = 0
while True:
    if mem[3776]>=41:
        break
    '''
    loc[98] = mem[1152+4*mem[3776]]
    
    loc[102] = mem[2336+4*mem[3776]]
    
    mem[1552+4*loc[102]] = loc[98]
    '''

In [ ]:
for i in range(41):
    x = mem[2336+4*i]
    mem[1552+4*x] = mem[1152+4*i]

In [ ]:
# 最后一段 label 13/12


mem[1552]
    
mem[5400]

两串相等，则 return 1

In [ ]:
"stk"



In [11]:
for i in range(len(mem)):
    if mem[i]:
        print(i,mem[i])

1952 22
1956 65
1960 61
1964 31
1968 87
1972 19
1976 38
1980 95
1984 4
1992 54
1996 3
2000 55
2004 5
2008 9
2012 53
2016 11
2020 45
2024 67
2028 14
2032 40
2036 91
2040 42
2044 29
2048 18
2052 62
2056 39
2060 68
2064 88
2068 60
2072 83
2076 47
2080 58
2084 44
2088 63
2092 6
2096 20
2100 94
2104 90
2108 70
2112 23
2116 93
2120 43
2124 82
2128 1
2132 37
2136 41
2140 32
2144 80
2148 33
2152 85
2156 73
2160 86
2164 76
2168 35
2172 66
2176 12
2180 16
2184 78
2188 24
2192 7
2196 52
2200 17
2204 10
2208 25
2212 84
2216 74
2220 81
2224 59
2228 75
2232 2
2236 48
2240 27
2244 49
2248 26
2252 21
2256 57
2260 56
2264 92
2268 69
2272 8
2276 13
2280 89
2284 15
2288 50
2292 28
2296 30
2300 46
2304 64
2308 79
2312 51
2316 77
2320 34
2324 72
2328 36
2332 71
3776 96
4986 41
5400 46
5404 113
5408 126
5412 48
5416 51
5420 81
5424 75
5428 76
5432 78
5436 83
5440 112
5444 34
5448 115
5452 54
5456 65
5460 81
5464 116
5468 69
5472 87
5476 60
5480 61
5484 77
5488 78
5492 118
5496 57
5500 40
5504 90
5508 77
551